# KGOntologies Example N#1
## AcmeBank Stafflist

Load a data file containing members of staff and the departments in which they are employed and process it into a marked-up knowledge-graph for visualisation. 



In [1]:
import os, sys

srcdir=r'..'
if srcdir not in sys.path:
    sys.path.append(srcdir)

from IPython.display import display, Image, HTML
import pandas as pd
import src.graphvisutils_gravis as graphvisutils_gravis 
import src.graphloader as graphloader
import src.queryaugment as queryaugment

import src.visgui as gui

import gravis as gv

import rdflib
from rdflib import URIRef, Literal, Namespace
import uuid
from hashlib import md5
import json

from functools import partial

/home/tomk/.local/share/virtualenvs/kgontologies-24Yr4f6_/lib/python3.10/site-packages/gravis/_internal/plotting/template_system.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as _pkg_resources


In [2]:
# Locate and read the data file
data_df = pd.read_excel("../data/AcmeBankStaffList.xlsx", na_values='None')

In [3]:
# Load and bind associated ontologies for additional data enrichment
kgdmcar_g = rdflib.Graph()
kgdmcar_g.parse ('../ontologies/kgdmcar.owl', format='xml')

kgnaming_g = rdflib.Graph()
kgnaming_g.parse ('../ontologies/kgnaming.owl', format='xml')

ontology_graph = rdflib.Graph(bind_namespaces="rdflib")
for t in kgnaming_g.triples((None, None, None)):
    ontology_graph.add(t)

for t in kgdmcar_g.triples((None, None, None)):
    ontology_graph.add(t)

In [4]:
# Convert the data into a raw graph format
rawg=graphloader.rdflib_graph_from_dataframe(data_df[0:500])

# Open a serialisation config file and convert the raw graph into triples describing entities, relations and literals
serial_config = json.load(open("../serialisations/acmestaff_serialisation.json", "r"))
load_graph  = graphloader.process_anonymous_data_graph(rawg, serial_config, data_namespace="http://data#")
# Bind some appropriate namespaces to the resulting subgraph object
namespace_dict = {  "kgnam" : Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#"),
                    "dmcar" : Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgdmcar#"),
                    "acme" : Namespace("http://www.acmebank.org/")}
graphloader.bind_namespaces(load_graph, namespace_dict)


In [5]:

test_ui = gui.gui_visualisation_control(load_graph)
test_ui.control

{rdflib.term.URIRef('http://www.acmebank.org/EndDate'): True, rdflib.term.URIRef('http://www.acmebank.org/Gender'): True, rdflib.term.URIRef('http://www.acmebank.org/HasRole'): True, rdflib.term.URIRef('http://www.acmebank.org/Name'): True, rdflib.term.URIRef('http://www.acmebank.org/StaffID'): True, rdflib.term.URIRef('http://www.acmebank.org/StartDate'): True, rdflib.term.URIRef('http://www.acmebank.org/WorksFor'): True, rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#FullyQualifiedName'): True, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'): True, rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'): True}
set()
set()


In [8]:
# Run a SPARQL Query to Retrieve only Active Staff Members (i.e. those with an assigned role)
# Note that this query is only returning nodes that meed certain conditions, the resulting
# resultset is not in s,p,o form, just contains a collection of node-uris.

active_staff_q = """
PREFIX acme: <http://www.acmebank.org/>

SELECT ?s ?q
WHERE 
    {
    
    {SELECT ?s ?q
    WHERE { 
    ?s a ?type.
    ?s acme:HasRole ?q.
    ?q a acme:Role.
    VALUES ?type {acme:Person} .
    }}

UNION

    {SELECT ?s
    WHERE { 
    ?s a ?type.
    VALUES ?type {acme:Role} .
    }}

UNION
    
    {SELECT ?s
    WHERE { 
    ?s a ?type.
    VALUES ?type {acme:Department} .
    }}

}


"""


In [9]:
# Run the query, then augment it with the get_x options, this allows for simpler workflow and easier sparql queries to be performed
# Augmentation results in a new subgraph, which is driven from the parameters
subgraph=queryaugment.augment_sparql_to_graph(load_graph, 
                          active_staff_q, 
                          get_types=True, 
                          get_labels=True, 
                          get_literals=True, 
                          get_cross_links=True, 
                          sparql_option=False)

In [10]:
# Bind some appropriate namespaces to the resulting subgraph object
graphloader.bind_namespaces(subgraph, namespace_dict)

In [12]:
subgraph_ui = gui.gui_visualisation_control(subgraph)
subgraph_ui.control

{rdflib.term.URIRef('http://www.acmebank.org/EndDate'): True, rdflib.term.URIRef('http://www.acmebank.org/Gender'): True, rdflib.term.URIRef('http://www.acmebank.org/HasRole'): True, rdflib.term.URIRef('http://www.acmebank.org/Name'): True, rdflib.term.URIRef('http://www.acmebank.org/StaffID'): True, rdflib.term.URIRef('http://www.acmebank.org/StartDate'): True, rdflib.term.URIRef('http://www.acmebank.org/WorksFor'): True, rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#FullyQualifiedName'): True, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'): True, rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'): True}
set()
set()
